<a href="https://colab.research.google.com/github/Seth-msum/Senior-Seminar-Project/blob/main/Basic_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#For the next steps I will need to edit the intents.json file which is stored
#in my onedrive cloud folder

In [1]:
#Mounting the Google Drive

from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/ChatBot'

Mounted at /content/drive


In [17]:
#Importing Relevant Libraries

import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#Loading the Dataset: intents.json

data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

In [5]:
#4 Creating data_X and data_Y

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow model/ vocabulary for tags
data_X = [] #For storing each pattern
data_Y = [] #For storing tag corresponding to each pattern in data_X
#Iterating over all the intents

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_X.append(pattern)
    data_Y.append(intent["tag"])


  if intent["tag"] not in classes:
    classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

words = sorted(set(words))
classes = sorted(set(classes))

In [7]:
#5 Text to Numbers
training = []
out_empty = [0] * len(classes)

for idx, doc in enumerate(data_X):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)

  output_row = list(out_empty)
  output_row[classes.index(data_Y[idx])] = 1
  training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [20]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0,5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0,5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               7552      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
Total params: 16848 (65.81 KB)
Trainable params: 16848 (65.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2

In [13]:
#7 Preprocessing the Input

def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
    return result

In [21]:
#8 Interacting with the chatbot
print("Press 0 if you dont't want to chat with our ChatBot")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)


Press 0 if you dont't want to chat with our ChatBot
hi
1/1 [==============================] - 0s 64ms/step
Good morning!
how are you
1/1 [==============================] - 0s 14ms/step
Very good and you?
I am good
1/1 [==============================] - 0s 14ms/step
That is perfect!
What is your name?
1/1 [==============================] - 0s 13ms/step
My job is to make you feel like everything is okay.
that is not what I asked
1/1 [==============================] - 0s 14ms/step
Hi!
no that does not make sense
1/1 [==============================] - 0s 15ms/step
I hope you will feel better !
What?
1/1 [==============================] - 0s 21ms/step
That is perfect!
0
